In [1]:
from Recommendation.recommendation_utils import get_ai_client_requirements

get_ai_client_requirements("Growth sutra is a LLP company knwon for GTM strategies","lient want to increase his sales by 10%")

"---\n### ✅ Refined Client Requirements\n\n**Project Title:** Growth Acceleration Program for [Client Name]\n\n**Project Goal:** To achieve a minimum 10% year-over-year increase in sales revenue within the next 12 months.  This will be measured by comparing Q4 2024 revenue against Q4 2023 revenue.\n\n**Project Scope:** This project will encompass a comprehensive review of [Client Name]'s current Go-to-Market (GTM) strategy, including but not limited to: market analysis, competitive landscape assessment, sales process optimization, lead generation strategies, sales enablement, and marketing alignment.  Growth Sutra will leverage its expertise in GTM strategy development and implementation to deliver tailored solutions.\n\n**Deliverables:**\n\n* **Phase 1 (Months 1-3):**  GTM Strategy Audit & Roadmap Development –  Comprehensive assessment of existing GTM processes, identification of key areas for improvement, and development of a detailed roadmap with prioritized initiatives.  Deliverab

In [1]:
from Recommendation.recommendation_utils import get_ai_business_priorities

result = get_ai_business_priorities("CEO")

/home/shreyank/anaconda3/envs/adk/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'title': 'Sustainable Revenue Growth & Market Share Expansion', 'icon': '📈'}, {'title': 'Digital Transformation & Operational Excellence', 'icon': '⚙️'}, {'title': 'Talent Acquisition & Retention in a Competitive Market', 'icon': '🧑\u200d💼'}]


In [2]:
type(result)

list

In [ ]:
from datetime import datetime
datetime.strftime()

In [1]:
from Recommendation.recommendation_utils import get_ai_proj_sepc_recommendations

from Recommendation.prompts import *
result = get_ai_proj_sepc_recommendations(effort_prompt,"Nothing","nothing")

```python
effort_data = {
    "Business Analysis": "**Business Analysis (120-160 hours)** • Requirements gathering and documentation\n• Process mapping and workflow analysis\n• Stakeholder management and communication\n• User story creation and prioritization\n",
    "Technical Development": "**Technical Development (400-600 hours)** • Frontend and backend development\n• Database design and implementation\n• API development and integration\n• Third-party library integration\n",
    "Testing & QA": "**Testing & QA (80-120 hours)** • Test planning and test case creation\n• Manual and automated testing execution\n• Bug fixing and regression testing\n• Performance and security testing\n",
    "Deployment & Support": "**Deployment & Support (40-80 hours)** • Deployment planning and execution\n• User training and documentation\n• Post-launch support and maintenance\n• System monitoring and performance tuning\n"
}
```
<class 'dict'>
{'Business Analysis': '**Business Analysis (120-160 hours)**

In [1]:
from PresentationWriting.src.main import get_presentation
from Client.client_dataclass import ClientData
from Seller.seller import SellerTabState

client = ClientData(enterprise_logo="https://static.wixstatic.com/media/cb6b3d_5c8f2b020ebe48b69bc8c163cc480156~mv2.png/v1/fill/w_60,h_60,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/GrowthSutra%20Logo.png")
seller = SellerTabState(enterprise_logo='https://benori.com/assets/web/images/Benori_Logo.svg')
# state = get_presentation(client,seller,[])
# from PresentationWriting.src.pres import *
# from PresentationWriting.src.utils import *
# colors = get_color_from_image(client.enterprise_logo) 
# generate_modern_presentation(filename  = 'output.txt',logo_url=client.enterprise_logo,logo_url_2 = seller.enterprise_logo ,output_format='pdf')


/home/shreyank/anaconda3/envs/adk/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from PresentationWriting.src.pres import *
from PresentationWriting.src.utils import *
colors = get_color_from_image(client.enterprise_logo) 
generate_modern_presentation(filename  = 'output.txt',logo_url=client.enterprise_logo,logo_url_2 = seller.enterprise_logo ,output_format='pdf')

📋 Found main title for cover: title of the sales proposal
📄 Added section: title of the sales proposal
📄 Added section: Executive Summary
📄 Added section: Understanding Your Needs
📄 Added section: Proposed Solution
📄 Added section: Scope of Work / Project Breakdown
📄 Added section: Deliverables
📄 Added section: Timeline
📄 Added section: Investment / Pricing
📄 Added section: Our Team / About Us
📄 Added section: Case Studies / Success Stories
📄 Added section: Terms and Conditions
📄 Added final section: Call to Action
✅ Parsed 12 sections with main title: 'title of the sales proposal'
✅ Generated HTML proposal: output_proposal.html


In [1]:
from PIL import Image, ImageDraw, ImageFont

def create_text_image(text, filename="text_image.png"):
    """
    Create a clean, professional image with black text on white background
    """
    
    # Try to get a professional font
    try:
        # Try professional fonts first
        font = ImageFont.truetype("arial.ttf", 32)
    except:
        try:
            font = ImageFont.truetype("Arial.ttf", 32)  # Windows
        except:
            try:
                font = ImageFont.truetype("Helvetica.ttc", 32)  # Mac
            except:
                try:
                    font = ImageFont.truetype("DejaVuSans-Bold.ttf", 32)  # Linux
                except:
                    font = ImageFont.load_default()
    
    # Create temporary image to measure text
    temp_img = Image.new('RGB', (1, 1), 'white')
    temp_draw = ImageDraw.Draw(temp_img)
    
    # Get text dimensions
    bbox = temp_draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    # Create final image with some padding
    padding = 50
    img_width = text_width + (padding * 2)
    img_height = text_height + (padding * 2)
    
    # Create clean white background
    img = Image.new('RGB', (img_width, img_height), 'white')
    draw = ImageDraw.Draw(img)
    
    # Draw text in strong black, centered
    x = padding
    y = padding
    draw.text((x, y), text, font=font, fill='black')
    
    # Save image
    img.save(filename, 'PNG', quality=100)
    print(f"✓ Image saved: {filename}")


create_text_image("benori")

✓ Image saved: text_image.png


In [6]:
l.keys()

dict_keys(['a', 'b'])

In [ ]:
import streamlit as st
from streamlit_extras.stylable_container import stylable_container
import time
from Client.client import client_tab,validate_client_mandatory_fields
from Seller.seller import seller_tab
from ProjectSpecification.project_spec import proj_specification_tab
from Generate_proposal.proposal_generator import generate_tab
from Client.client import setup_logging
from Client.client_dataclass import ClientData
from Seller.seller import SellerTabState

# Initialize session state variables
if 'client_data_from_tab' not in st.session_state:
    st.session_state.client_data_from_tab = None

if 'seller_data_from_tab' not in st.session_state:
    st.session_state.seller_data_from_tab = None

if 'project_specs_from_tab' not in st.session_state:
    st.session_state.project_specs_from_tab = None

# Initialize locked tabs state
if 'locked_tabs' not in st.session_state:
    st.session_state.locked_tabs = set()

# Initialize highest reached tab
if 'highest_reached_tab' not in st.session_state:
    st.session_state.highest_reached_tab = 0

def get_sample_extracted_text():
    return """Key Requirements Extracted:

• Project Type: Enterprise Software Development
• Timeline: 6-8 months
• Budget Range: $150,000 - $200,000
• Team Size: 5-7 developers
• Technologies: React, Node.js, PostgreSQL
• Deployment: AWS Cloud Infrastructure
• Security: SOC 2 compliance required
• Integration: Salesforce, HubSpot APIs
• Support: 24/7 monitoring and maintenance

Additional Notes:
- Client prefers agile methodology
- Weekly progress reports required
- UAT phase: 4 weeks
- Go-live date: Q3 2024"""

def refresh_all_data():
    """Clear all session state and form data"""
    # Clear all session state variables
    keys_to_clear = [
        'client_name_input', 'url_selector', 'pain_points', 'pain_points_extracted',
        'pain_points_placeholder', 'editable_content_area', 'pain_points_summary',
        'selected_roles', 'selected_priorities', 'problem_statement'
    ]
    
    for key in keys_to_clear:
        if key in st.session_state:
            del st.session_state[key]
    
    # Clear locked tabs and reset navigation
    st.session_state.locked_tabs = set()
    st.session_state.highest_reached_tab = 0
    st.session_state.active_tab = 0
    
    # Clear confirmation states
    confirmation_keys = [key for key in st.session_state.keys() if key.startswith('show_confirmation_')]
    for key in confirmation_keys:
        del st.session_state[key]
    
    # Clear any other dynamic keys (role and priority related)
    keys_to_remove = []
    for key in st.session_state.keys():
        if (key.startswith('role_edit_input_') or 
            key.startswith('remove_role_btn_') or 
            key.startswith('priority_checkbox_')):
            keys_to_remove.append(key)
    
    for key in keys_to_remove:
        del st.session_state[key]
    
    st.success("All data has been cleared!")
    st.rerun()

def validate_seller_mandatory_fields():
    """Validate seller mandatory fields"""
    # Add your seller validation logic here
    # For now, returning True as placeholder
    return True

def validate_project_mandatory_fields():
    """Validate project specification mandatory fields"""
    # Add your project validation logic here
    # For now, returning True as placeholder
    return True

def show_validation_popup(missing_tab_name, missing_fields=None):
    """Show validation error popup"""
    st.error(f"⚠️ Please complete all mandatory fields in {missing_tab_name} tab first!")
    if missing_fields:
        st.error(f"Missing required fields: {missing_fields}")

def get_validation_function(tab_index):
    """Get validation function for a specific tab"""
    if tab_index == 0:
        return validate_client_mandatory_fields
    elif tab_index == 1:
        return validate_seller_mandatory_fields
    elif tab_index == 2:
        return validate_project_mandatory_fields
    else:
        return lambda: True

def is_tab_accessible(tab_index):
    """Check if a tab is accessible based on validation"""
    if tab_index == 0:
        return True
    elif tab_index == 1:
        return validate_client_mandatory_fields()
    elif tab_index == 2:
        return validate_client_mandatory_fields() and validate_seller_mandatory_fields()
    elif tab_index == 3:
        return (validate_client_mandatory_fields() and 
                validate_seller_mandatory_fields() and 
                validate_project_mandatory_fields())
    return False

def show_lock_confirmation_popup(tab_index):
    """Show popup-style confirmation dialog for locking a tab"""
    tab_names = ["Client Information", "Seller Information", "Project Specifications", "Generate Proposal"]
    
    # Create popup-style confirmation dialog
    with stylable_container(
        f"confirmation_popup_{tab_index}",
        css_styles="""
        div[data-testid="stBlock"] {
            position: fixed !important;
            top: 50% !important;
            left: 50% !important;
            transform: translate(-50%, -50%) !important;
            z-index: 9999 !important;
            background: white !important;
            border: 2px solid #e74c3c !important;
            border-radius: 15px !important;
            padding: 25px !important;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.3) !important;
            min-width: 400px !important;
            max-width: 500px !important;
        }
        div[data-testid="stBlock"]:before {
            content: '' !important;
            position: fixed !important;
            top: 0 !important;
            left: 0 !important;
            width: 100vw !important;
            height: 100vh !important;
            background: rgba(0, 0, 0, 0.5) !important;
            z-index: -1 !important;
        }
        """,
    ):
        # Warning icon and title
        st.markdown(
            f"""
            <div style="text-align: center; margin-bottom: 20px;">
                <span style="font-size: 48px; color: #e74c3c;">⚠️</span>
                <h3 style="color: #e74c3c; margin: 10px 0;">Important Notice</h3>
            </div>
            """, 
            unsafe_allow_html=True
        )
        
        # Warning message
        st.markdown(
            f"""
            <div style="text-align: center; margin-bottom: 25px;">
                <p style="font-size: 16px; color: #2c3e50; line-height: 1.5;">
                    Moving to the next tab will <strong>lock</strong> the <strong>'{tab_names[tab_index]}'</strong> tab.<br>
                    You <strong>cannot modify</strong> it later.
                </p>
                <p style="font-size: 14px; color: #7f8c8d; margin-top: 15px;">
                    Are you sure you want to proceed?
                </p>
            </div>
            """, 
            unsafe_allow_html=True
        )
        
        # Action buttons
        col1, col2 = st.columns(2, gap="medium")
        
        with col1:
            with stylable_container(
                f"confirm_btn_{tab_index}",
                css_styles="""
                button {
                    background-color: #e74c3c !important;
                    color: white !important;
                    border: none !important;
                    border-radius: 8px !important;
                    padding: 10px 20px !important;
                    font-weight: bold !important;
                    width: 100% !important;
                    transition: all 0.3s ease !important;
                }
                button:hover {
                    background-color: #c0392b !important;
                    transform: translateY(-1px) !important;
                }
                """,
            ):
                if st.button("✅ Yes, Continue", key=f"confirm_lock_{tab_index}"):
                    # Clear confirmation state
                    if f"show_confirmation_{tab_index}" in st.session_state:
                        del st.session_state[f"show_confirmation_{tab_index}"]
                    # Lock the current tab
                    st.session_state.locked_tabs.add(tab_index)
                    # Move to next tab
                    st.session_state.active_tab = tab_index + 1
                    st.session_state.highest_reached_tab = max(st.session_state.highest_reached_tab, st.session_state.active_tab)
                    st.rerun()
        
        with col2:
            with stylable_container(
                f"cancel_btn_{tab_index}",
                css_styles="""
                button {
                    background-color: #95a5a6 !important;
                    color: white !important;
                    border: none !important;
                    border-radius: 8px !important;
                    padding: 10px 20px !important;
                    font-weight: bold !important;
                    width: 100% !important;
                    transition: all 0.3s ease !important;
                }
                button:hover {
                    background-color: #7f8c8d !important;
                    transform: translateY(-1px) !important;
                }
                """,
            ):
                if st.button("❌ No, Stay Here", key=f"cancel_lock_{tab_index}"):
                    # Clear confirmation state
                    if f"show_confirmation_{tab_index}" in st.session_state:
                        del st.session_state[f"show_confirmation_{tab_index}"]
                    st.rerun()
    
    return True

def navigate_to_next_tab():
    """Navigate to the next tab with validation and locking"""
    current_tab = st.session_state.active_tab
    
    # Get validation function for current tab
    validation_func = get_validation_function(current_tab)
    
    if not validation_func():
        tab_names = ["Client Information", "Seller Information", "Project Specifications", "Generate Proposal"]
        show_validation_popup(tab_names[current_tab])
        return
    
    # If tab is already locked, just navigate
    if current_tab in st.session_state.locked_tabs:
        if current_tab < 3:
            st.session_state.active_tab = current_tab + 1
            st.session_state.highest_reached_tab = max(st.session_state.highest_reached_tab, st.session_state.active_tab)
            st.rerun()
        return
    
    # If this is the last tab, don't show confirmation
    if current_tab >= 3:
        return
    
    # Show confirmation dialog
    confirmation_key = f"show_confirmation_{current_tab}"
    st.session_state[confirmation_key] = True
    st.rerun()

def navigate_to_previous_tab():
    """Navigate to the previous tab"""
    if st.session_state.active_tab > 0:
        st.session_state.active_tab -= 1
        st.rerun()

def get_button_text(direction, current_tab):
    """Get button text with tab names"""
    tab_names = ["Client Information", "Seller Information", "Project Specifications", "Generate Proposal"]
    
    if direction == "next":
        if current_tab < 3:
            return f"Next: {tab_names[current_tab + 1]} ➡️"
        else:
            return "Next ➡️"
    else:  # previous
        if current_tab > 0:
            return f"⬅️ Previous: {tab_names[current_tab - 1]}"
        else:
            return "⬅️ Previous"

def is_tab_locked(tab_index):
    """Check if a tab is locked"""
    return tab_index in st.session_state.locked_tabs

# NOTE: Add this line at the very top of your main script (before any other Streamlit commands):
st.set_page_config(page_title="CoXPRT", page_icon="Images/gs_logo.png", layout="wide")
        
from main_css import *
st.markdown(app_css, unsafe_allow_html=True)

# Initialize session state for active tab - ENSURE CLIENT TAB IS DEFAULT
if 'active_tab' not in st.session_state:
    st.session_state.active_tab = 0

# Tab buttons
tab_names = ["Client Information", "Seller Information", "Project Specifications", "Generate Proposal"]

# Create tab buttons with stylable containers for #599cd4 active state
cols = st.columns(4, gap="large")
for i, tab_name in enumerate(tab_names):
    with cols[i]:
        is_active = (i == st.session_state.active_tab)
        
        # Determine if tab should be clickable based on validation
        tab_enabled = is_tab_accessible(i)
        
        # Add lock indicator to tab name if locked
        display_name = tab_name
        if is_tab_locked(i):
            display_name = f"🔒 {tab_name}"
        
        # Use stylable_container to make active tab #599cd4
        if is_active and tab_enabled:
            with stylable_container(
                f"active_tab_{i}",
                css_styles="""
                button {
                    background-color: #599cd4 !important;
                    color: white !important;
                    border: 2px solid #4a8bc2 !important;
                    font-weight: bold !important;
                    transition: all 0.3s ease !important;
                    box-shadow: 0 4px 8px rgba(89, 156, 212, 0.3) !important;
                }
                button:hover {
                    background-color: #4a8bc2 !important;
                    color: white !important;
                    transform: translateY(-2px) !important;
                    box-shadow: 0 6px 12px rgba(89, 156, 212, 0.4) !important;
                }
                button:focus {
                    background-color: #3b7ab0 !important;
                    color: white !important;
                    outline: none !important;
                }
                """,
            ):
                if st.button(display_name, key=f"tab_{i}", use_container_width=True):
                    st.session_state.active_tab = i
                    st.rerun()
        elif tab_enabled:
            with stylable_container(
                f"inactive_tab_{i}",
                css_styles="""
                button {
                    background-color: #6c757d !important;
                    color: white !important;
                    border: 1px solid #5a6268 !important;
                    transition: all 0.3s ease !important;
                }
                button:hover {
                    background-color: #5a6268 !important;
                    color: white !important;
                    transform: translateY(-1px) !important;
                }
                """,
            ):
                if st.button(display_name, key=f"tab_{i}", use_container_width=True):
                    st.session_state.active_tab = i
                    st.rerun()
        else:
            # Disabled tab styling
            with stylable_container(
                f"disabled_tab_{i}",
                css_styles="""
                button {
                    background-color: #e9ecef !important;
                    color: #6c757d !important;
                    border: 1px solid #dee2e6 !important;
                    cursor: not-allowed !important;
                    opacity: 0.6 !important;
                }
                """,
            ):
                st.button(display_name, key=f"tab_{i}", use_container_width=True, disabled=True)

# Handle confirmation dialogs - POPUP STYLE
current_tab = st.session_state.active_tab
confirmation_key = f"show_confirmation_{current_tab}"

if confirmation_key in st.session_state and st.session_state[confirmation_key]:
    show_lock_confirmation_popup(current_tab)
    st.stop()  # Stop execution to show only the popup

# Set is_active flag for current tab
st.session_state.is_active = True

# Show lock status message for locked tabs
if is_tab_locked(current_tab):
    st.info(f"🔒 This tab is locked. You cannot modify the data in this tab.")

# Content area with validation-aware tab switching
if st.session_state.active_tab == 0:
    logger = setup_logging()
    # Pass locked status to the tab
    st.session_state.client_data_from_tab = client_tab(st, logger, is_locked=is_tab_locked(0))

elif st.session_state.active_tab == 1:
    # Double-check validation before showing seller tab
    if validate_client_mandatory_fields():
        st.session_state.seller_data_from_tab = seller_tab(is_locked=is_tab_locked(1))
    else:
        st.session_state.active_tab = 0  # Force back to client tab
        show_validation_popup("Client Information")
        st.rerun()

elif st.session_state.active_tab == 2:
    # Check both client and seller validations
    if not validate_client_mandatory_fields():
        st.session_state.active_tab = 0  # Force back to client tab
        show_validation_popup("Client Information")
        st.rerun()
    elif not validate_seller_mandatory_fields():
        st.session_state.active_tab = 1  # Force back to seller tab
        show_validation_popup("Seller Information")
        st.rerun()
    else:
        print(st.session_state.client_data_from_tab, st.session_state.seller_data_from_tab)
        st.session_state.project_specs_from_tab = proj_specification_tab(
            st.session_state.client_data_from_tab, 
            st.session_state.seller_data_from_tab,
            is_locked=is_tab_locked(2)
        )

else:  # Generate Proposal Tab
    # Check all validations
    if not validate_client_mandatory_fields():
        st.session_state.active_tab = 0
        show_validation_popup("Client Information")
        st.rerun()
    elif not validate_seller_mandatory_fields():
        st.session_state.active_tab = 1
        show_validation_popup("Seller Information")
        st.rerun()
    elif not validate_project_mandatory_fields():
        st.session_state.active_tab = 2
        show_validation_popup("Project Specifications")
        st.rerun()
    else:
        generate_tab(
            st.session_state.client_data_from_tab,
            st.session_state.seller_data_from_tab,
            st.session_state.project_specs_from_tab
        )

# Bottom navigation buttons with enhanced styling
col1, col2, col3 = st.columns(3, gap="large")

# Previous Button
with col1:
    is_first_tab = (st.session_state.active_tab == 0)
    prev_button_text = get_button_text("previous", st.session_state.active_tab)
    
    if is_first_tab:
        with stylable_container(
            "prev_button_disabled",
            css_styles="""
            button {
                background-color: #e9ecef !important;
                color: #6c757d !important;
                border: 1px solid #dee2e6 !important;
                cursor: not-allowed !important;
                opacity: 0.6 !important;
                font-weight: bold !important;
            }
            """,
        ):
            st.button(prev_button_text, key="prev_btn", use_container_width=True, disabled=True)
    else:
        with stylable_container(
            "prev_button",
            css_styles="""
            button {
                background-color: #6c757d !important;
                color: white !important;
                border: 1px solid #5a6268 !important;
                font-weight: bold !important;
                transition: all 0.3s ease !important;
            }
            button:hover {
                background-color: #5a6268 !important;
                color: white !important;
                transform: translateY(-1px) !important;
            }
            button:active {
                background-color: #599cd4 !important;
                border: 2px solid #4a8bc2 !important;
                transform: translateY(0px) !important;
                box-shadow: 0 2px 4px rgba(89, 156, 212, 0.4) !important;
            }
            """,
        ):
            if st.button(prev_button_text, key="prev_btn", use_container_width=True):
                navigate_to_previous_tab()

# Refresh Button
with col2:
    with stylable_container(
        "refresh_button",
        css_styles="""
        button {
            background-color: #6c757d !important;
            color: white !important;
            border: 1px solid #5a6268 !important;
            font-weight: bold !important;
            transition: all 0.3s ease !important;
        }
        button:hover {
            background-color: #5a6268 !important;
            color: white !important;
            transform: translateY(-1px) !important;
        }
        button:active {
            background-color: #599cd4 !important;
            border: 2px solid #4a8bc2 !important;
            transform: translateY(0px) !important;
            box-shadow: 0 2px 4px rgba(89, 156, 212, 0.4) !important;
        }
        """,
    ):
        if st.button("🔄 Refresh All Data", key="refresh_btn", use_container_width=True):
            refresh_all_data()

# Next Button
with col3:
    is_last_tab = (st.session_state.active_tab == 3)
    next_button_text = get_button_text("next", st.session_state.active_tab)
    
    if is_last_tab:
        with stylable_container(
            "next_button_disabled",
            css_styles="""
            button {
                background-color: #e9ecef !important;
                color: #6c757d !important;
                border: 1px solid #dee2e6 !important;
                cursor: not-allowed !important;
                opacity: 0.6 !important;
                font-weight: bold !important;
            }
            """,
        ):
            st.button(next_button_text, key="next_btn", use_container_width=True, disabled=True)
    else:
        with stylable_container(
            "next_button",
            css_styles="""
            button {
                background-color: #6c757d !important;
                color: white !important;
                border: 1px solid #5a6268 !important;
                font-weight: bold !important;
                transition: all 0.3s ease !important;
            }
            button:hover {
                background-color: #5a6268 !important;
                color: white !important;
                transform: translateY(-1px) !important;
            }
            button:active {
                background-color: #599cd4 !important;
                border: 2px solid #4a8bc2 !important;
                transform: translateY(0px) !important;
                box-shadow: 0 2px 4px rgba(89, 156, 212, 0.4) !important;
            }
            """,
        ):
            if st.button(next_button_text, key="next_btn", use_container_width=True):
                navigate_to_next_tab()

In [4]:
from SalesProposalWriting.src.main import *
from Client.client_dataclass import ClientData
client = ClientData()

get_presentation(client,client,"")

2025-07-10 09:35:29.248 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 09:35:29.249 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-07-10 09:35:29.250 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 09:35:29.251 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 09:35:29.252 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 09:35:29.254 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

📋 Found main title for cover: Title of the sales proposal
📄 Added section: Title of the sales proposal
📄 Added section: Executive Summary
📄 Added section: Introduction
📄 Added section: About GrowthSutra
📄 Added section: Understanding Your Needs
📄 Added section: Proposed Solution (Revenue Architect & XPRT Co-Pilots)
📄 Added section: Scope of Work / Project Breakdown
📄 Added section: Timeline & Milestones
📄 Added section: Pricing & Payment Terms
📄 Added section: Next Steps
📄 Added section: Who We Are
📄 Added final section: What We Do
✅ Parsed 12 sections with main title: 'Title of the sales proposal'
✅ Generated HTML proposal: /home/shreyank/GS/GS_Sales_Proposal/SalesProposalsGenerated/benori_proposal.html


'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Title of the sales proposal</title>\n    <meta name="description" content="Professional Sales Proposal - Title of the sales proposal">\n    <style>\n        @import url(\'https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&family=Playfair+Display:wght@400;500;600;700&display=swap\');\n        \n        :root {\n            --primary: #1a365d;\n            --secondary: #2d3748;\n            --accent: #3182ce;\n            --success: #38a169;\n            --background: #ffffff;\n            --surface: #f7fafc;\n            --text: #2d3748;\n            --text-light: #718096;\n            --border: #e2e8f0;\n            --shadow-sm: 0 1px 2px 0 rgba(0, 0, 0, 0.05);\n            --shadow-md: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);\n            --shadow-lg: 0 10px 15px -3px rgba(0